# MySQL + SPARK

![my-saprk](spark.png)

Este cuaderno muestra la conexión de SparkSQL con una base de datos relacional, el en caso presente utilizamos MySQL

### Lectura de una tabla de mysql

## Las fuentes de información que se ocupan en el ejercicio son datos de un concurso de kaggle sobre BIMBO

In [1]:
%%bash

jps


11363 SecondaryNameNode
11539 Master
11044 NameNode
20778 Jps
11612 Worker
11725 LivyServer
11167 DataNode


#### Conexion Spark BdRelacional

In [2]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,None,pyspark3,idle,,,✔


SparkSession available as 'spark'.
<SparkContext master=spark://hostsaxsa:7077 appName=livy-session-6>

In [3]:
from pyspark.sql import Row

In [4]:
df_ss = spark.read.format("jdbc").options(
    url="jdbc:mysql://localhost/bimbodb",
    driver="com.mysql.jdbc.Driver",
    dbtable="sample_submission",
    user="saxsauser",
    password="ggsaXsa").load()

In [5]:
df_ss.describe().show()

+-------+------------------+--------------------+
|summary|                id|   Demanda_uni_equil|
+-------+------------------+--------------------+
|  count|           3500000|             3500000|
|   mean|3500478.1217282857|            6.999998|
| stddev|2020719.6132133713|0.003741657386773...|
|    min|                 0|                   0|
|    max|           6999247|                   7|
+-------+------------------+--------------------+

In [6]:
df_ss.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Demanda_uni_equil: integer (nullable = true)

In [7]:
df_ss.createOrReplaceTempView('sample_submission')

In [8]:
%%sql
SELECT Demanda_uni_equil, COUNT(Demanda_uni_equil) AS Apariciones FROM sample_submission GROUP BY Demanda_uni_equil

Output()

In [9]:
df_t = spark.read.format("jdbc").options(
    url="jdbc:mysql://localhost/bimbodb",
    driver="com.mysql.jdbc.Driver",
    dbtable="test",
    user="saxsauser",
    password="ggsaXsa").load()

In [10]:
column = df_t.columns
column

['id', 'Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Cliente_ID', 'Producto_ID']

In [11]:
for col in column:
    df_t.select(col).describe().show()

+-------+-----------------+
|summary|               id|
+-------+-----------------+
|  count|          3500000|
|   mean|3498583.557187714|
| stddev|2020471.762325858|
|    min|                0|
|    max|          6999250|
+-------+-----------------+

+-------+------------------+
|summary|            Semana|
+-------+------------------+
|  count|           3500000|
|   mean|         10.494304|
| stddev|0.4999990555828539|
|    min|                 0|
|    max|                11|
+-------+------------------+

+-------+-----------------+
|summary|       Agencia_ID|
+-------+-----------------+
|  count|          3500000|
|   mean|2504.166604857143|
| stddev| 4009.29680864309|
|    min|                0|
|    max|            25759|
+-------+-----------------+

+-------+------------------+
|summary|          Canal_ID|
+-------+------------------+
|  count|           3500000|
|   mean|1.4022545714285715|
| stddev|1.5149633380090768|
|    min|                 0|
|    max|                11|


In [12]:
df_sel = df_t.select('Agencia_ID', 'Cliente_ID', 'Producto_ID')
df_sel.show(20)

+----------+----------+-----------+
|Agencia_ID|Cliente_ID|Producto_ID|
+----------+----------+-----------+
|      1420|   1875998|      34213|
|      4010|   1661843|      30549|
|      1375|   1234559|      43307|
|      1531|    158133|      35303|
|      1544|   2309256|       2233|
|      1629|   4527410|      41938|
|      1470|   4435970|      36610|
|      1122|   2055889|       1109|
|      2032|   4200375|       1212|
|      1216|   4522344|      35305|
|      1315|   4573216|      37361|
|      1955|    573512|      43064|
|      1338|    198309|      43274|
|      2644|    273971|       4270|
|      2014|    276970|      45143|
|      2011|    126719|       1160|
|      4063|   2451950|      43344|
|      1221|     58826|      30532|
|      1124|   1590017|       1109|
|      1954|   1101612|       1146|
+----------+----------+-----------+
only showing top 20 rows

In [13]:
df_sel.select('Producto_ID').distinct().count()

1426

In [14]:
df_wri = df_sel.groupby('Producto_ID').count().sort('count', ascending=False)
df_wri.show()

+-----------+-----+
|Producto_ID|count|
+-----------+-----+
|       2233|94763|
|       1242|85109|
|       1250|77308|
|      35305|72972|
|       1240|69454|
|       1146|68716|
|       1109|65984|
|       1232|64003|
|       1125|62965|
|       1278|61439|
|      43069|61207|
|      43285|58953|
|       1284|56728|
|      30532|55688|
|      35651|52796|
|       1212|51713|
|       1238|50749|
|       1150|50480|
|       1309|48998|
|      37361|48087|
+-----------+-----+
only showing top 20 rows

### Escribir una tabla en mysql

In [15]:
df_wri.write.format("jdbc").options(
    url="jdbc:mysql://localhost/bimbodb",
    driver="com.mysql.jdbc.Driver",
    dbtable="tabla_escrita",
    user="saxsauser",
    password="ggsaXsa").mode('append').save()

In [16]:
%%bash

hdfs dfs -ls /

Found 1 items
drwxr-xr-x   - saxsa supergroup          0 2019-08-05 15:43 /FuentesCurso


In [17]:
df_wri.write.parquet('/consulta_parquet')

In [21]:
%%bash

hdfs dfs -ls /consulta_parquet

Found 150 items
-rw-r--r--   1 saxsa supergroup          0 2019-08-05 23:38 /consulta_parquet/_SUCCESS
-rw-r--r--   1 saxsa supergroup        654 2019-08-05 23:37 /consulta_parquet/part-00000-fc69a552-8608-4a3a-bec3-caf99c434222-c000.snappy.parquet
-rw-r--r--   1 saxsa supergroup        645 2019-08-05 23:37 /consulta_parquet/part-00001-fc69a552-8608-4a3a-bec3-caf99c434222-c000.snappy.parquet
-rw-r--r--   1 saxsa supergroup        644 2019-08-05 23:37 /consulta_parquet/part-00002-fc69a552-8608-4a3a-bec3-caf99c434222-c000.snappy.parquet
-rw-r--r--   1 saxsa supergroup        644 2019-08-05 23:37 /consulta_parquet/part-00003-fc69a552-8608-4a3a-bec3-caf99c434222-c000.snappy.parquet
-rw-r--r--   1 saxsa supergroup        643 2019-08-05 23:37 /consulta_parquet/part-00004-fc69a552-8608-4a3a-bec3-caf99c434222-c000.snappy.parquet
-rw-r--r--   1 saxsa supergroup        643 2019-08-05 23:37 /consulta_parquet/part-00005-fc69a552-8608-4a3a-bec3-caf99c434222-c000.snappy.parquet
-rw-r--r--   1 saxsa 

In [18]:
df_t.createOrReplaceTempView('test')

In [19]:
%%sql
SELECT Cliente_ID, COUNT(CLiente_ID) AS Apariciones FROM test GROUP BY Cliente_ID ORDER BY Apariciones DESC LIMIT 60

Output()